<a href="https://colab.research.google.com/github/dolmani38/Summary2/blob/main/multi-discriminator%20GAN%200830.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Discriminator GAN


ABSTRACT

----


#4. Implementation


## 4.1 기본 설정...

In [1]:
if True:
    from google.colab import drive
    drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#!pip install keybert
!pip3 install transformers
!pip3 install sentence-transformers

#!pip install sentence-transformers

In [3]:
import matplotlib.pyplot as plt
# set seeds for reproducability
from numpy.random import seed
#seed(1)

import pandas as pd
import numpy as np
import string, os 

import urllib.request
import nltk
nltk.download('punkt')

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [5]:
# Print iterations progress
class ProgressBar:

    def __init__(self,total=20, prefix = '', suffix = '', decimals = 1, length = 20, fill = '|', printEnd = "\r"):
        self.total = total
        self.prefix = prefix
        self.suffix = suffix
        self.decimals = decimals
        self.length = length
        self.fill = fill
        self.printEnd = printEnd
        self.ite = 0
        self.back_filledLength = 0

    def printProgress(self,iteration, text):
        self.ite += iteration
        percent = ("{0:." + str(self.decimals) + "f}").format(100 * (self.ite / float(self.total)))
        filledLength = int(self.length * self.ite // self.total)
        bar = self.fill * filledLength + '.' * (self.length - filledLength)
        if filledLength > self.back_filledLength or percent == 100:
            print(f'\r{self.prefix} |{bar}| {percent}% {self.suffix}  {text}', end="", flush=True)
            # Print New Line on Complete
            if self.ite == self.total: 
                print()
        self.back_filledLength = filledLength    

In [6]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

##4.2 Grammar Discriminator Class

In [7]:
from transformers import BertTokenizer, BertTokenizerFast,AutoTokenizer, BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset, random_split

import time
import random
import datetime

# 간단한 전처리
def clean_text(txt):
    txt = txt.replace('\n',' ')
    txt = txt.replace('\r',' ')    
    txt = txt.replace('=','')
    txt = txt.replace('\"','')   
    txt = txt.replace('\'','')
    #txt = txt.replace(',','')
    txt = txt.replace('..','')
    txt = txt.replace('...','')
    txt = txt.replace(' .','.')
    txt = txt.replace('.','. ')
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')           
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')             
    return txt.strip()

def shuffling(txt):
    txt_list = txt.split(' ')
    random.shuffle(txt_list)
    return ' '.join(txt_list)

def collect_training_dataset_for_grammar_discriminator(sentences_dataset):

    sentences = []
    labels = []

    for txtss in sentences_dataset:
        txtss = clean_text(txtss)
        txts = txtss.strip().split('.')
        for txt in txts:  
            txt = txt.strip()
            if len(txt) > 10:
                #ko_grammar_dataset.append([txt,1])
                txt = txt.replace('.','')
                tf = random.choice([True,False])
                # 정상 또는 비정상 둘중에 하나만 데이터셋에 추가
                if (tf):
                    sentences.append(txt) # '.'의 위치를 보고 True, False를 판단 하기 땜에...
                    labels.append(1)
                else:
                    sentences.append(shuffling(txt))
                    labels.append(0)

    return sentences,labels

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

class Grammar_Discriminator:


    def __init__(self, pretraoned_kobert_model_name='kykim/bert-kor-base', input_dir=None):

        if input_dir is None:
            self.tokenizer = BertTokenizerFast.from_pretrained(pretraoned_kobert_model_name)
            self.discriminator = BertForSequenceClassification.from_pretrained(
                                    pretraoned_kobert_model_name, # Use the 12-layer BERT model, with an uncased vocab.
                                    num_labels = 2, # The number of output labels--2 for binary classification.
                                                    # You can increase this for multi-class tasks.   
                                    output_attentions = False, # Whether the model returns attentions weights.
                                    output_hidden_states = False, # Whether the model returns all hidden-states.
                                )            
        else:
            self.__load_model(input_dir)




    def set_dataset(self, sentences,labels):
        # Print the original sentence.
        print(' Original: ', sentences[0])

        # Tokenize all of the sentences and map the tokens to thier word IDs.
        input_ids = []
        attention_masks = []

        # For every sentence...
        for sent in sentences:
            # `encode_plus` will:
            #   (1) Tokenize the sentence.
            #   (2) Prepend the `[CLS]` token to the start.
            #   (3) Append the `[SEP]` token to the end.
            #   (4) Map tokens to their IDs.
            #   (5) Pad or truncate the sentence to `max_length`
            #   (6) Create attention masks for [PAD] tokens.
            encoded_dict = self.tokenizer.encode_plus(
                                sent,                      # Sentence to encode.
                                add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                                max_length = 64,           # Pad & truncate all sentences.
                                pad_to_max_length = True,
                                return_attention_mask = True,   # Construct attn. masks.
                                return_tensors = 'pt',     # Return pytorch tensors.
                                truncation = True,
                        )
            
            # Add the encoded sentence to the list.    
            input_ids.append(encoded_dict['input_ids'])
            
            # And its attention mask (simply differentiates padding from non-padding).
            attention_masks.append(encoded_dict['attention_mask'])

        # Convert the lists into tensors.
        input_ids = torch.cat(input_ids, dim=0)
        attention_masks = torch.cat(attention_masks, dim=0)
        labels = torch.tensor(labels)

        # Print sentence 0, now as a list of IDs.
        print('Original: ', sentences[0])
        print('Token IDs:', input_ids[0])

        # Training & Validation Split
        # Divide up our training set to use 90% for training and 10% for validation.

        # Combine the training inputs into a TensorDataset.
        dataset = TensorDataset(input_ids, attention_masks, labels)

        # Create a 90-10 train-validation split.

        # Calculate the number of samples to include in each set.
        train_size = int(0.9 * len(dataset))
        val_size = len(dataset) - train_size

        # Divide the dataset by randomly selecting samples.
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

        print('{:>5,} training samples'.format(train_size))
        print('{:>5,} validation samples'.format(val_size))

        # The DataLoader needs to know our batch size for training, so we specify it 
        # here. For fine-tuning BERT on a specific task, the authors recommend a batch 
        # size of 16 or 32.
        self.batch_size = 32

        # Create the DataLoaders for our training and validation sets.
        # We'll take training samples in random order. 
        self.train_dataloader = DataLoader(
                    train_dataset,  # The training samples.
                    sampler = RandomSampler(train_dataset), # Select batches randomly
                    batch_size = self.batch_size # Trains with this batch size.
                )

        # For validation the order doesn't matter, so we'll just read them sequentially.
        self.validation_dataloader = DataLoader(
                    val_dataset, # The validation samples.
                    sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
                    batch_size = self.batch_size # Evaluate with this batch size.
                )        



    def train(self,epochs=4):
        # Tell pytorch to run this model on the GPU.
        self.discriminator.cuda()

        # Get all of the model's parameters as a list of tuples.
        params = list(self.discriminator.named_parameters())

        print('The BERT model has {:} different named parameters.\n'.format(len(params)))

        print('==== Embedding Layer ====\n')

        for p in params[0:5]:
            print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

        print('\n==== First Transformer ====\n')

        for p in params[5:21]:
            print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

        print('\n==== Output Layer ====\n')

        for p in params[-4:]:
            print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))  

        # Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
        # I believe the 'W' stands for 'Weight Decay fix"
        self.optimizer = AdamW(self.discriminator.parameters(),
                        lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                        eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                        )

        # Number of training epochs. The BERT authors recommend between 2 and 4. 
        # We chose to run for 4, but we'll see later that this may be over-fitting the
        # training data.
        #epochs = 2

        # Total number of training steps is [number of batches] x [number of epochs]. 
        # (Note that this is not the same as the number of training samples).
        total_steps = len(self.train_dataloader) * epochs

        # Create the learning rate scheduler.
        scheduler = get_linear_schedule_with_warmup(self.optimizer, 
                                                    num_warmup_steps = 0, # Default value in run_glue.py
                                                    num_training_steps = total_steps)
            
        # This training code is based on the `run_glue.py` script here:
        # https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

        # Set the seed value all over the place to make this reproducible.
        seed_val = 42

        random.seed(seed_val)
        np.random.seed(seed_val)
        torch.manual_seed(seed_val)
        torch.cuda.manual_seed_all(seed_val)

        # We'll store a number of quantities such as training and validation loss, 
        # validation accuracy, and timings.
        training_stats = []

        # Measure the total training time for the whole run.
        total_t0 = time.time()

        # For each epoch...
        for epoch_i in range(0, epochs):
            
            # ========================================
            #               Training
            # ========================================
            
            # Perform one full pass over the training set.

            print("")
            print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
            print('Training...')

            # Measure how long the training epoch takes.
            t0 = time.time()

            # Reset the total loss for this epoch.
            total_train_loss = 0

            # Put the model into training mode. Don't be mislead--the call to 
            # `train` just changes the *mode*, it doesn't *perform* the training.
            # `dropout` and `batchnorm` layers behave differently during training
            # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
            self.discriminator.train()

            # For each batch of training data...
            for step, batch in enumerate(self.train_dataloader):

                # Progress update every 40 batches.
                if step % 40 == 0 and not step == 0:
                    # Calculate elapsed time in minutes.
                    elapsed = format_time(time.time() - t0)
                    
                    # Report progress.
                    print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(self.train_dataloader), elapsed))

                # Unpack this training batch from our dataloader. 
                #
                # As we unpack the batch, we'll also copy each tensor to the GPU using the 
                # `to` method.
                #
                # `batch` contains three pytorch tensors:
                #   [0]: input ids 
                #   [1]: attention masks
                #   [2]: labels 
                b_input_ids = batch[0].to(device)
                b_input_mask = batch[1].to(device)
                b_labels = batch[2].to(device)

                # Always clear any previously calculated gradients before performing a
                # backward pass. PyTorch doesn't do this automatically because 
                # accumulating the gradients is "convenient while training RNNs". 
                # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
                self.discriminator.zero_grad()        

                # Perform a forward pass (evaluate the model on this training batch).
                # The documentation for this `model` function is here: 
                # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
                # It returns different numbers of parameters depending on what arguments
                # arge given and what flags are set. For our useage here, it returns
                # the loss (because we provided labels) and the "logits"--the model
                # outputs prior to activation.
                outputs = self.discriminator(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask, 
                                    labels=b_labels)
                loss, logits = outputs.loss, outputs.logits
                # Accumulate the training loss over all of the batches so that we can
                # calculate the average loss at the end. `loss` is a Tensor containing a
                # single value; the `.item()` function just returns the Python value 
                # from the tensor.
                total_train_loss += loss.item()

                # Perform a backward pass to calculate the gradients.
                loss.backward()

                # Clip the norm of the gradients to 1.0.
                # This is to help prevent the "exploding gradients" problem.
                torch.nn.utils.clip_grad_norm_(self.discriminator.parameters(), 1.0)

                # Update parameters and take a step using the computed gradient.
                # The optimizer dictates the "update rule"--how the parameters are
                # modified based on their gradients, the learning rate, etc.
                self.optimizer.step()

                # Update the learning rate.
                scheduler.step()

            # Calculate the average loss over all of the batches.
            avg_train_loss = total_train_loss / len(self.train_dataloader)            
            
            # Measure how long this epoch took.
            training_time = format_time(time.time() - t0)

            print("")
            print("  Average training loss: {0:.2f}".format(avg_train_loss))
            print("  Training epcoh took: {:}".format(training_time))
                
            # ========================================
            #               Validation
            # ========================================
            # After the completion of each training epoch, measure our performance on
            # our validation set.

            print("")
            print("Running Validation...")

            t0 = time.time()

            # Put the model in evaluation mode--the dropout layers behave differently
            # during evaluation.
            self.discriminator.eval()

            # Tracking variables 
            total_eval_accuracy = 0
            total_eval_loss = 0
            nb_eval_steps = 0

            # Evaluate data for one epoch
            for batch in self.validation_dataloader:
                
                # Unpack this training batch from our dataloader. 
                #
                # As we unpack the batch, we'll also copy each tensor to the GPU using 
                # the `to` method.
                #
                # `batch` contains three pytorch tensors:
                #   [0]: input ids 
                #   [1]: attention masks
                #   [2]: labels 
                b_input_ids = batch[0].to(device)
                b_input_mask = batch[1].to(device)
                b_labels = batch[2].to(device)
                
                # Tell pytorch not to bother with constructing the compute graph during
                # the forward pass, since this is only needed for backprop (training).
                with torch.no_grad():        

                    # Forward pass, calculate logit predictions.
                    # token_type_ids is the same as the "segment ids", which 
                    # differentiates sentence 1 and 2 in 2-sentence tasks.
                    # The documentation for this `model` function is here: 
                    # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
                    # Get the "logits" output by the model. The "logits" are the output
                    # values prior to applying an activation function like the softmax.
                    outputs = self.discriminator(b_input_ids, 
                                        token_type_ids=None, 
                                        attention_mask=b_input_mask,
                                        labels=b_labels)
                loss, logits = outputs.loss, outputs.logits
                # Accumulate the validation loss.
                total_eval_loss += loss.item()

                # Move logits and labels to CPU
                logits = logits.detach().cpu().numpy()
                label_ids = b_labels.to('cpu').numpy()

                # Calculate the accuracy for this batch of test sentences, and
                # accumulate it over all batches.
                total_eval_accuracy += flat_accuracy(logits, label_ids)
                

            # Report the final accuracy for this validation run.
            avg_val_accuracy = total_eval_accuracy / len(self.validation_dataloader)
            print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

            # Calculate the average loss over all of the batches.
            avg_val_loss = total_eval_loss / len(self.validation_dataloader)
            
            # Measure how long the validation run took.
            validation_time = format_time(time.time() - t0)
            
            print("  Validation Loss: {0:.2f}".format(avg_val_loss))
            print("  Validation took: {:}".format(validation_time))

            # Record all statistics from this epoch.
            training_stats.append(
                {
                    'epoch': epoch_i + 1,
                    'Training Loss': avg_train_loss,
                    'Valid. Loss': avg_val_loss,
                    'Valid. Accur.': avg_val_accuracy,
                    'Training Time': training_time,
                    'Validation Time': validation_time
                }
            )

        print("")
        print("Training complete!")

        print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
            

        return training_stats

    def save_model(self, output_dir = './model_save/'):
        # Create output directory if needed
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        print("Saving model to %s" % output_dir)

        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = self.discriminator.module if hasattr(self.discriminator, 'module') else self.discriminator  # Take care of distributed/parallel training
        model_to_save.save_pretrained(output_dir)
        self.tokenizer.save_pretrained(output_dir)

        # Good practice: save your training arguments together with the trained model
        # torch.save(args, os.path.join(output_dir, 'training_args.bin'))

    def __load_model(self, input_dir = './drive/MyDrive/Colab Notebooks/summary/en_grammar_check_model'):
        print('Loading BERT tokenizer...')
        self.tokenizer = BertTokenizerFast.from_pretrained(input_dir)
        self.discriminator = BertForSequenceClassification.from_pretrained(input_dir)

    def transfer_learning(self, sentences, train_for = True):
        
        input_ids = []
        attention_masks = []

        # For every sentence...
        for sent in sentences:
            # `encode_plus` will:
            #   (1) Tokenize the sentence.
            #   (2) Prepend the `[CLS]` token to the start.
            #   (3) Append the `[SEP]` token to the end.
            #   (4) Map tokens to their IDs.
            #   (5) Pad or truncate the sentence to `max_length`
            #   (6) Create attention masks for [PAD] tokens.
            encoded_dict = self.tokenizer.encode_plus(
                                sent,                      # Sentence to encode.
                                add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                                max_length = 64,           # Pad & truncate all sentences.
                                pad_to_max_length = True,
                                return_attention_mask = True,   # Construct attn. masks.
                                return_tensors = 'pt',     # Return pytorch tensors.
                                truncation = True,
                        )
            # Add the encoded sentence to the list.    
            input_ids.append(encoded_dict['input_ids'])

            # And its attention mask (simply differentiates padding from non-padding).
            attention_masks.append(encoded_dict['attention_mask'])
        
        if train_for:
            b_labels = torch.ones(len(sentences),dtype=torch.long).to(device)
        else:
            b_labels = torch.zeros(len(sentences),dtype=torch.long).to(device)
        #print(b_labels)
        # Convert the lists into tensors.
        input_ids = torch.cat(input_ids, dim=0).to(device)
        attention_masks = torch.cat(attention_masks, dim=0).to(device)    
        #if str(discriminator1.device) == 'cpu':
        #    pass
        #else:
        #    input_ids = input_ids.to(device)
        #    attention_masks = attention_masks.to(device)        

        outputs = self.discriminator(input_ids, 
                            token_type_ids=None, 
                            attention_mask=attention_masks, 
                                labels=b_labels)

        #print(outputs)
        #return torch.sigmoid(outputs[0][:,1])
        #return outputs[0][:,1]
        return outputs['loss'], outputs['logits']


# 문법 학습을 위한 데이터셋 구성

In [8]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/summary/korean_news_corpus.csv')
df

,title,contents
0,문 대통령 변창흠 국토장관 사의표명 사실상 수용,정만호 국민소통수석이 12일 오후 청와대 춘추관 대브리핑룸에서 변창흠 국토부 장관 ...
1,계급장 수여하는 문 대통령,(아산뉴스1) 이광호 기자 문재인 대통령이 12일 오후 충남 아산시 경찰대학에서 열...
2,계급장 수여하는 문 대통령,(아산뉴스1) 이광호 기자 문재인 대통령이 12일 오후 충남 아산시 경찰대학에서 열...
3,수상자 메달 걸어주는 문 대통령,(아산뉴스1) 이광호 기자 문재인 대통령이 12일 오후 충남 아산시 경찰대학에서 열...
4,정몽구 서울아산병원에 50억 쾌척,인재 양성·소외 계층 지원 등 계획 “부친 질병·가난 악순환 끊기 원해 국내 최고 ...
...,...,...
140559,[건축과도시] 북한산을 캔버스 삼아. 미술관 또 하나의 작품이 되다,<은평구 진관동 사비나 미술관> 서울시 은평구 진관동에 자리잡은 사비나미술관. 삼각...
140560,조선후기 문인 김조순 별장 그린 옥호정도 첫 공개,국립중앙박물관 서화실 개편해 32점 새롭게 전시 옥호정도[국립중앙박물관 제공연합뉴스...
140561,안성 청룡사 대웅전에서 목재 곡자 발견,문화재청(청장 정재숙)의 국고보조와 기술지도로 안성시에서 시행하고 있는 안성 청룡사...
140562,156년전 ㄱ자 곡자 찾았다 안성 청룡사 기둥 밑에서,안성 청룡사 대웅전에서 발견된 곡자 【서울뉴시스】 이수지 기자 안성 청룡사 대웅전에...


In [9]:
df = df.dropna(axis=0)
df['contents'].count()

140536

In [10]:
df['contents'][0]

'정만호 국민소통수석이 12일 오후 청와대 춘추관 대브리핑룸에서 변창흠 국토부 장관 사의 표명 관련 브리핑을 하고 있다. 연합뉴스 문재인 대통령이 변창흠 국토교통부 장관의 사의 표명을 사실상 수용했다. 정만호 청와대 국민소통수석은 12일 오후 청와대 춘추관 대브리핑룸에서 브리핑을 통해 변창흠 국토교통부 장관이 이날 한국토지주택공사(LH) 직원 등의 3기 신도시 땅 투기 의혹과 관련해 문재인 대통령에게 사의를 표명했다고 전했다. 문 대통령은 변 장관의 사의 표명에 책임지는 모습을 보일 수밖에 없다고 밝혔다. 이어 다만 2·4 대책의 차질없는 추진이 매우 중요하다며 변 장관 주도로 추진한 공공주도형 공급대책과 관련된 입법의 기초작업까지는 마무리해야 한다고 말했다. 이는 문 대통령이 사실상 변 장관의 사의를 수용한 것으로 해석된다. 이에 앞서 변창흠 국토교통부 장관은 LH 땅 투기 의혹 사건과 관련한 책임론에 대해 “자리에 연연하지 않는다”면서 “(청와대의) 결정에 따르겠다”고 말했다. 변 장관은 이날 국회 국토교통위원회 전체회의에 참석해 “LH 사태로 국민들이 걱정하는 부분을 해소할 수 있게 최대한 대안을 만들고 LH가 근본적으로 다시 태어날 수 있도록 책임지고 추진하겠다”고 언급하고 “그 역할이 충분하다고 평가되지 못했을 때 언제든지 자리에 연연하지 않고 결정에 따르겠다”고 말했다. 특히 경찰에 수사의뢰된 20명 중 11명은 변 장관이 LH 사장 재임 시절 땅 투기에 나선 것으로 드러나면서 LH 사장 출신인 변 장관에 대한 책임론이 대두되던 상황이었다. 전창훈 기자 jch@busan. com ▶ 네이버에서 부산일보 구독하기 클릭! ▶ 부산닷컴 회원가입. 회원 전환하면 부산일보 지면보기 무료이벤트 ▶ 부산일보 홈 바로가기'

In [11]:
import re
import sys

import nltk
nltk.download('punkt')

# 간단한 전처리
def clean_text(txt):
    txt = txt.replace('\n',' ')
    txt = txt.replace('\r',' ')    
    txt = txt.replace('=','')
    txt = txt.replace('\"','')   
    txt = txt.replace('\'','')
    txt = txt.replace('”','')
    txt = txt.replace('“','')
    txt = txt.replace('’','')
    #txt = txt.replace(',','')
    txt = txt.replace('..','')
    txt = txt.replace('...','')
    #txt = txt.replace('.','. ')
    txt = txt.replace('.','. ')
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')           
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')             
    return txt.strip()
    
# 검사...
pattens = ["[34569][0-9]{3}[\;.\;-\; ][0-9]{4}[\;.\;-\; ][0-9]{4}[\;.\;-\; ][0-9]{4}",
           "[0-9]{2,3}[\:\s\;.\;,\;-;)][0-9]{3,4}[\:\s\;.\;,\;-][0-9]{4}",
           "[0-9]{1}[0-9]{1}[\W]?[0-1]{1}[0-9]{1}[\W]?[0-3]{1}[\W]?[0-9]{1}[\W]?[1-4]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}",
           "[0-9]{2}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{3}[\:\s\;.\;,\;-]([0-9]{5,6}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{6}[\:\s\;.\;,\;-][0-9]{5}|[0-9]{2,3}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{7}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{4,6}[\:\s\;.\;,\;-][0-9]|[0-9]{5}[\:\s\;.\;,\;-][0-9]{3}[\:\s\;.\;,\;-][0-9]{2}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{5}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{4}[\:\s\;.\;,\;-][0-9]{4}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{6}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{7})|[0-9]{4}[\:\s\;.\;,\;-]([0-9]{3}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{6}[\:\s\;.\;,\;-][0-9])|[0-9]{5}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{6}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{5,6}"
           ]

filters = []
for p in pattens:
    filters.append(re.compile(p))

sentences = []
df = df.dropna(axis=0)
cnt = df['contents'].count()
#print('Total row count:',cnt)
i=0
for raw_text in df['contents']:
    i=i+1
    try:
        if i%100 == 0:
            percent = ("{0:.2f}").format(100 * (i / float(cnt)))
            print(f'\r {percent}% {i}/{str(cnt)}', end="", flush=True)

        docs = nltk.sent_tokenize(clean_text(raw_text))
        for txt in docs:
            if txt.find('▶') > -1 or txt.find('@') > -1 or txt.find('ⓒ') > -1: 
                pass
            else:
                txt = txt.strip()
                if any(chr.isdigit() for chr in txt) :
                    pass
                else:
                    sentences.append(txt)
    except KeyboardInterrupt as ki:
        raise ki        
    except:
        pass #print("Unexpected error:", sys.exc_info()[0])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
 3.56% 5000/140536

KeyboardInterrupt: ignored

In [12]:
import json  
import zipfile  

d1 = None  
data = None  
with zipfile.ZipFile("/content/drive/MyDrive/Colab Notebooks/summary/data/사설잡지_2.train_original.json.zip", "r") as z:
    with z.open('train_original.json') as f:  
        data = f.read()  
        d1 = json.loads(data.decode("utf-8"))

In [ ]:
for i in range(len(d1)):
    for txt in d1[i]['article_original']:
        sentences.append(txt)

In [13]:
d2 = None  
data = None  
with zipfile.ZipFile("/content/drive/MyDrive/Colab Notebooks/summary/data/신문기사_2.train_original.json.zip", "r") as z:
    with z.open('train_original.json') as f:  
        data = f.read()  
        d2 = json.loads(data.decode("utf-8"))

In [ ]:
' '.join(d2[1]['article_original'])

In [ ]:
for i in range(len(d2)):
    for txt in d1[2]['article_original']:
        sentences.append(txt)

In [ ]:
import re
import sys
import io

#텍스트 정제(전처리)
def cleanText(readData):
    #텍스트에 포함되어 있는 특수 문자 제거
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》◆◇●🎧○▲\t―△━▷]', '', readData)
    return text.lower()

In [ ]:
c_sentences = []
for sentence in sentences:
    s = cleanText(sentence)
    c = len(s.split())
    if c >= 3 and c < 10 and s.find('재배포') < 0 and s.find('기자') < 0  and s.find('유투브') < 0 and s.find('www') < 0 and s.find('com') < 0 and s.find('접속하기') < 0 and s.find('http') < 0 and s.find('뉴스') < 0 and s.find('일보') < 0 :
        if s.endswith(('다','요')):
            c_sentences.append(s.strip())

In [ ]:
len(c_sentences)

In [ ]:
c_sentences[:100]

In [ ]:
ko_sentences_dataset = c_sentences

# 문법 discriminator의 학습

In [14]:
use_pretrained_model = True

if use_pretrained_model:
    #g_discriminator = Grammar_Discriminator(input_dir = '/content/drive/MyDrive/Colab Notebooks/summary/model_save')
    g_discriminator = Grammar_Discriminator(input_dir = '/content/drive/MyDrive/Colab Notebooks/summary/ko_grammar_model3')
else:
    sentences,labels = collect_training_dataset_for_grammar_discriminator(ko_sentences_dataset)
    print(len(sentences))
    g_discriminator = Grammar_Discriminator()
    g_discriminator.set_dataset(sentences,labels)
    g_discriminator.train(epochs=1)
    g_discriminator.save_model(output_dir='/content/drive/MyDrive/Colab Notebooks/summary/ko_grammar_model3')

Loading BERT tokenizer...


In [15]:
if False: ## 추가적인 fine-tuning
    g_discriminator.train(epochs=10)
    g_discriminator.save_model(output_dir='/content/drive/MyDrive/Colab Notebooks/summary/ko_grammar_model3')

In [16]:
txt = ['최근 날씨가 포근해지면서 산을 찾는 사람들도 늘고 있는데요','서비스를 음원 플랫폼 스포티파이가 국내 론칭한다']
g_discriminator.discriminator.to(device)
g_discriminator.transfer_learning(txt)

(tensor(4.3092, device='cuda:0', grad_fn=<NllLossBackward>),
 tensor([[-4.0743,  4.2727],
         [ 4.2365, -4.3814]], device='cuda:0', grad_fn=<AddmmBackward>))

##4.3 Static similarity discriminator class

In [17]:
from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from scipy.misc import electrocardiogram
import scipy


class Similarity_Discriminator:
    '''
    _instance = None
    _embedder = None
    def __new__(cls,pre_trained_model_name='stsb-roberta-large'):
        if cls._instance is None:
            print('Creating Similarity_Discriminator object')
            cls._instance = super(Similarity_Discriminator, cls).__new__(cls)
            # Put any initialization here.
            cls._embedder = SentenceTransformer(pre_trained_model_name)
        return cls._instance

    '''

    def __init__(self,pre_trained_model_name='xlm-r-large-en-ko-nli-ststb'):
        print('Creating Similarity_Discriminator object')
        # Put any initialization here.
        self._embedder = SentenceTransformer(pre_trained_model_name)  
        #self.cos = nn.CosineSimilarity(dim=1, eps=1e-6)

    def encode(self,texts):
        return self._embedder.encode(texts,show_progress_bar=False)

    def similarity(self, query_text, org_text_emb):
        queries = nltk.sent_tokenize(query_text)
        query_embeddings = self._embedder.encode(queries,show_progress_bar=False)
        #query_embeddings = self._embedder.encode(queries,show_progress_bar=False)
        #print(queries)
        #print(org_text_emb)
        
        if len(query_embeddings) == 0:
            return 0.0

        cos_scores = scipy.spatial.distance.cdist(query_embeddings, org_text_emb, "cosine")
        similarity_score = 1.0 - np.mean(np.min(cos_scores,axis=0))
        '''
        for query, query_embedding in zip(queries, query_embeddings):
            distances = scipy.spatial.distance.cdist([query_embedding], [org_text_emb], "cosine")[0]
            results = zip(range(len(distances)), distances)
            for idx, distance in results:
                scores.append(1-distance)
        '''
        return similarity_score  
 

###4.3.1 한국어 문장 유사도 pre-trained model 적용

In [18]:
s_discriminator = Similarity_Discriminator()
#s_discriminator = Similarity_Discriminator()

Creating Similarity_Discriminator object


# 4.4 Document source class

## 두 문장을 합치는 rule 변환기... --> 이거... ML로 나중에 바꿔야...

In [19]:
combine_matching_table = {}

combine_matching_table['어요.'] = '고'
combine_matching_table['지요.'] = '고'
combine_matching_table['답니다.'] = '고'
combine_matching_table['보거라.'] = '봐,'
combine_matching_table['간단다.'] = '가니,'
combine_matching_table['돼.'] = '되,'
combine_matching_table['해.'] = '하며,'
combine_matching_table['다.'] = '고'


In [20]:
def combine_sentence(txt):
    for c in combine_matching_table.keys():
        if txt.endswith(c):
            txt = txt.replace(c,combine_matching_table[c])
    return txt

In [21]:

conjunction_table = ['그러던','그래서','그러나','그런데','그리고','그랬더니','그러니까','하지만','그래서']

## 4.4.2 source class 구현

In [22]:


class Source:

    def __init__(self,org_text):
        self.org_text = org_text

    def __crean_text(self, txt):
        txt = txt.replace('\n',' ')
        txt = txt.replace('\r',' ')    
        txt = txt.replace('=','')
        txt = txt.replace('\"','')   
        txt = txt.replace('\'','')
        txt = txt.replace(',','')
        txt = txt.replace('..','')
        txt = txt.replace('...','')
        txt = txt.replace(' .','.')
        txt = txt.replace('.','. ')
        txt = txt.replace('  ',' ')
        txt = txt.replace('  ',' ')    
        txt = txt.replace('  ',' ')   
        txt = txt.replace('  ',' ')           
        txt = txt.replace('  ',' ')
        txt = txt.replace('  ',' ')    
        txt = txt.replace('  ',' ')   
        txt = txt.replace('  ',' ')           
        return txt.strip()

    def set_key_rate(self,s_discriminator):
        #self.full_text = self.__crean_text(self.full_text.strip())
        self.org_text = self.__crean_text(self.org_text.strip())
        print('-'*50)
        print(self.org_text)
        print('-'*50)
        self.org_sentences = np.array(nltk.sent_tokenize(self.org_text))
        for i,sents in enumerate(self.org_sentences):
            for cj in conjunction_table: 
                if sents.startswith(cj):
                    self.org_sentences[i] = sents[len(cj):].strip()

        #self.full_sentences = np.array(nltk.sent_tokenize(self.full_text))
        
        #self.org_term_set = (' ' + self.org_text + ' ').split(' ')
        self.org_term_set = (' '.join(self.org_sentences)).strip().split(' ')
        self.org_source_length = len(self.org_term_set)
        self.term_table = {}
        self.seps = []
        #morp_table = {}

        for index, word in zip(range(len(self.org_term_set)),self.org_term_set):
            self.term_table[index] = word
            if word.endswith(('.','?')):
                self.seps.append(index)
                if self.org_source_length - 1 == index:
                    pass
                else:
                    self.term_table[index] = combine_sentence(word)

        #print(self.term_table.values())
        #print('------------------------------------------------------------------')

        self.s_discriminator = s_discriminator
        # 원문의 embedding...
        self.org_text_emb = self.s_discriminator.encode(self.org_sentences)
        #self.full_text_emb = self.s_discriminator.encode(self.full_sentences)
        #top_n = int(len(self.term_table) * comp_rate)
        #print('top_n',top_n)
        #self.story_peaks = [i+1 for i in range(top_n)]

    def get_org_sample(self, num):
        return self.org_sentences[np.random.choice(len(self.org_sentences), num)]

    def get_source_embedded_code(self):
        return self.org_text_emb

    def get_random_text(self,rate=0.7):
        cnt = int(len(self.term_table) * rate)
        a = list(self.term_table.keys())
        b = np.random.choice(a, cnt)
        c = [fruit for fruit in a if fruit not in b]
        txt = []
        for i in c:
            txt.append(self.term_table[i])
        return ' '.join(txt).strip(), hash(tuple(b))

In [23]:
txt = """
황금마차는 호박으로, 흰말은 생쥐로, 마부는 도마뱀으로 변하게 돼어요. 
그러니까 반드시 밤 열두시가 되기 전에 돌아와야 해요.
"""
s = Source(txt)
s.set_key_rate(s_discriminator)
s.get_random_text()

--------------------------------------------------
황금마차는 호박으로 흰말은 생쥐로 마부는 도마뱀으로 변하게 돼어요. 그러니까 반드시 밤 열두시가 되기 전에 돌아와야 해요.
--------------------------------------------------


('호박으로 흰말은 생쥐로 밤 열두시가 되기 전에 돌아와야', 6553471788267561388)

# 4.5 Generator class

In [24]:
from functools import reduce


# custom weights initialization called on netG and netD
'''
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        m.weight.data.normal_(0.0, 0.06)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(0.0, 0.05)
        m.bias.data.fill_(0)
'''
class Generator(nn.Module):
    """
        Simple Generator w/ MLP
    """
    '''
    def __init__(self, input_size=1024):
        super(Generator, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(input_size, input_size*2),
            nn.BatchNorm1d(input_size*2),
            nn.LeakyReLU(0.2),
            nn.Linear(input_size*2, input_size*3),
            nn.BatchNorm1d(input_size*3),
            nn.LeakyReLU(0.2),
            nn.Linear(input_size*3, input_size*3),
            nn.BatchNorm1d(input_size*3),
            nn.LeakyReLU(0.2),            
            nn.Linear(input_size*3, input_size*2),
            nn.BatchNorm1d(input_size*2),
            nn.LeakyReLU(0.2),
            nn.Linear(input_size*2, input_size),
            #nn.BatchNorm1d(term_length*4),
            nn.Tanh() # -1 ~ 1
        )
    '''
    
    def __init__(self, input_size=1024):
        super(Generator, self).__init__()
        l1 = nn.Linear(input_size, input_size*4)
        l1.weight.data.normal_(0.0, 0.03)
        bn = nn.BatchNorm1d(input_size*4)
        bn.weight.data.normal_(0.0, 0.05)
        bn.bias.data.fill_(0)        
        l2 = nn.Linear(input_size*4, input_size)
        l2.weight.data.normal_(0.01, 0.05)
        self.layer = nn.Sequential(
            l1,
            bn,
            nn.LeakyReLU(0.2),
            l2,
            #nn.BatchNorm1d(term_length*4),
            nn.Tanh() # -1 ~ 1
        )

    def forward(self, x, bias):
        #biased_noise = torch.randn(N,_NOISE_DIM)
        # stroy peak에 해당하는 term에게 평균값에 해당하는 bias를 추가 한다.
                 
        y_ = self.layer(x)
        y = torch.add(y_,bias)
        #y = nn.Sigmoid()(y)

        return y, y_


# multi-discriminator에 대한 Adaptive discriminant factor 를 구하기 위한 학습

ref : https://realpython.com/python-ai-neural-network/

colab 수식입력 : 

https://wikidocs.net/1679

https://en.wikipedia.org/wiki/Help:Displaying_a_formula#Formatting_using_TeX

Original GAN의 목적함수
$$ \min_{G}\max_{D} V(D,G) = E_{x\sim p_{data}(x)}[logD(x)] + E_{z\sim p_{z}(z)}[log(1-D(G(z)))] $$

Multi-Discriminator GAN은 discriminator가 각 목적에 의하여 여러개 (N개) 있다.
MDGAN의 목적함수

$$ \min_{G}\max_{D_{i\sim N}} V(D_{i\sim N},G) = \sum_{i=1}^N \{E_{x\sim p_{data}(x)}[logD_i(x)] + E_{z\sim p_{z}(z)}[log(1-D_i(G(z)))]\} $$

여기서

$$ L(D_i,G) =  E_{x\sim p_{data}(x)}[logD_i(x)] + E_{z\sim p_{z}(z)}[log(1-D_i(G(z)))] $$

이라하고 단순화 하면

$$ \min_{G}\max_{D_{i\sim N}} V(D_{i\sim N},G) = \sum_{i=1}^N L(D_i,G) $$

와 같이 된다.

문제점은 GAN의 특성상, 특정 Discriminator가 학습에 있어 지배적으로 loss 함수에 영향을 미치게 되어 각각의 Discriminator가 골고루 학습에 참여하지 못하고 의도하지 않은 결과를 만들게 된다. 이러한 문제점을 극복하기 위해 다음의 두가지 제안을 한다.

1) 목적함수에 각 Loss 에 대한 표준편차 (standard-deviation) 를 반영하여 각 Discriminator에 대한 Loss가 상호 유사한 수준을 유지하면 학습이 진행되도록 한다.

$$ \min_{G}\max_{D_{i\sim N}} V(D_{i\sim N},G) = \sum_{i=1}^N L(D_i,G) + STD_{i \sim N}(L(D_i,G))$$

2) 1)의 제안에 추가하여, 각 discriminator에 의한 loss를 제어하기 위해, adaptive discriminant factor (ADF) 를 적용하고, 학습의 진행 과정에서 이를 최적화 한다. 

$$ \min_{G}\max_{D_{i\sim N}} V(D_{i\sim N},G) = \sum_{i=1}^N f_iL(D_i,G) + STD_{i \sim N}(L(D_i,G))$$

여서서 

fi = adaptive discriminant factor for discriminator i 

중요한 것은, 학습과정에서 Li을 작게 (학습의 방향)하기 위해서는 fi는 역으로 커져야 한다. 그래야, 전체 Loss function에서 비중이 증대되어 더 적극적인 학습이 이루어 지게 된다. 따라서, fi의 최적화 방향은 기존의 gradient decent와 반대 방향이 되어야 한다.

$$ f_i^{t+1} = f_i^t + \alpha \frac{\partial V}{\partial f_i}$$

##4.6 Summarizer class (GAN training)

# 학습기...

In [25]:
import random
import math
import numpy as np
from scipy.special import expit
from transformers import get_linear_schedule_with_warmup

class SAM_Summarizer:

    def __init__(self,g_discriminator,s_discriminator):
        self.g_discriminator = g_discriminator
        #self.c_discriminator = c_discriminator
        self.s_discriminator = s_discriminator
        self.m = nn.Sigmoid()

    def ready(self,source):
        self.source = source  
        #self.source.analysis_frame_terms(self.s_discriminator)
        self.generator = Generator(input_size=self.source.org_source_length)
        #self.generator.apply(weights_init)
        return self

    def summarize(self,epochs=10,batch_size=1,learning_rate=2e-4, display = False):
        history = self.__train(epochs,batch_size,learning_rate,display)
        if display:
            plt.figure(figsize=(12, 6))
            plt.plot(history['gen_g_loss'],label='grammar loss')
            plt.plot(history['gen_l_loss'],label='compression loss')
            plt.plot(history['gen_s_loss'],label='similarity loss')

            plt.plot(history['total loss'],label='total loss')
            plt.plot(history['losses std'],label='standard deviation of losses')
            
            #if 'dis_loss' in history:
            #    plt.plot(history['dis_loss'],label='discriminator grammar loss')
            plt.legend()
            plt.show()

        return history

    # text의 생성 for torch
    def __text_gen2(self, p_txt, gen_length):
        gtext = []
        sorted_noise, i = torch.sort(p_txt, descending=True)
        order, i = torch.sort(i[:gen_length], descending=False)
        #print(len(order))
        #print(gen_length)
        assert len(order) == gen_length
        order = order.cpu().detach().numpy()
        for k in order:
            gtext.append((self.source.term_table[k],k))
        return gtext

    def __text_gen3(self, p_txt):
        gtext = []

        for order,p in enumerate(p_txt):
            if p > 0.0:
                gtext.append(self.source.term_table[order])
        return gtext

    def __text_gen4(self, p_txt):
        gtext = ""
        indexs = []
        for order,p in enumerate(p_txt):
            if p > 0.0:
                gtext += self.source.term_table[order] + ' '
                indexs.append(order)
        return gtext.strip(),indexs


    def __discrete_gradient(self,weights,use_gpu=False, verbose=0):
        fake_gen_out = torch.zeros(weights.shape).to(device)
        #fake_com_out = torch.zeros(weights.shape).to(device)
        fake_sim_out = torch.zeros(weights.shape).to(device)
        fake_len_out = torch.zeros(weights.shape).to(device) 

        #real_text = self.source.get_org_sample(weights.shape[0])
        fake_outs = []
        #real_outs = []
        apply_order = []
        for i, noise in enumerate(weights):
            #gtext = self.__text_gen2(noise,gen_length)
            gtext,tk = self.__text_gen4(noise)
            fake_outs.append(gtext)
            apply_order.append((i,tk))
  
        D_z_loss, fake_gmr_out=self.g_discriminator.transfer_learning(fake_outs,train_for = False)

        o_sim_out = []
        o_len_out = []
        for fake_text in fake_outs:
            o_sim_out.append(self.s_discriminator.similarity(fake_text,self.source.org_text_emb))

            l = ((1 - len(fake_text.split(' '))/self.source.org_source_length)-0.5) * 2

            o_len_out.append(l)
            #print(1 - len(fake_text.split(' '))/self.source.org_source_length)
            #o_len_out.append(-len(fake_text.split(' '))/self.source.org_source_length)
        
        
        for j, (i,tk) in enumerate(apply_order):

            try:
                '''
                a = torch.tanh( fake_gmr_out[j,1])
                if a > 0 :
                    fake_gen_out[:] = -0.5
                    fake_gen_out[i,tk] = a
                else:
                    fake_gen_out[:] = 0.5
                    fake_gen_out[i,tk] = a
                '''
                fake_gen_out[i,tk] = torch.tanh( fake_gmr_out[j,1])
                fake_sim_out[i,tk] = o_sim_out[j]
                fake_len_out[:] = o_len_out[j]
                fake_len_out[i,tk] = 0 #torch.tensor(fake_text_len/self.source.org_source_length).to(device)
                #fake_len_out[:] = o_len_out[j] #torch.tensor(fake_text_len/self.source.org_source_length).to(device)
                #print(o_len_out[j])
            except Exception as ex:
                print(j,i,tk)
                print(fake_gmr_out)
                raise ex

        return fake_gen_out, fake_sim_out, fake_len_out #fake_com_out, fake_sim_out #, D_z_loss, D_x_loss


    def __train(self, epochs=10,batch_size=10,learning_rate=2e-4,display = False):
        # In the Deepmind paper they use RMSProp however then Adam optimizer
        # improves training time
        #generator_optimizer = tf.keras.optimizers.Adam(1e-4)
        # This method returns a helper function to compute cross entropy loss
        #cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

        # Set the seed value all over the place to make this reproducible.
        seed_val = int(random.random()*100)

        random.seed(seed_val)
        np.random.seed(seed_val)
        torch.manual_seed(seed_val)
        torch.cuda.manual_seed_all(seed_val)
        
        criterion = nn.BCELoss()
        #D_opt = torch.optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))
        G_opt = AdamW(self.generator.parameters(),
                        lr = 2e-3, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                        eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                        )
        # Create the learning rate scheduler.
        scheduler = get_linear_schedule_with_warmup(G_opt, 
                                                    num_warmup_steps = 0, # Default value in run_glue.py
                                                    num_training_steps = epochs)
        
        #gen_length = len(self.source.story_peaks) + int(len(self.source.story_peaks)*self.frame_expansion_ratio)
        pb = ProgressBar(epochs,prefix='Train...')
        gen_gmr_loss_history = []
        gen_len_loss_history = []
        gen_sim_loss_history = []
        dis_loss_history = []    
        total_loss_history = []
        losses_std_history = []

        #model 들은 cuda로 보낸다.
        self.g_discriminator.discriminator.to(device)
        self.g_discriminator.discriminator.eval() # 학습하지 않는다...
        #self.c_discriminator.discriminator.to(device)
        #self.c_discriminator.discriminator.eval() # 학습하지 않는다...

        self.generator.to(device)       
        self.generator.train()

        #self.bias_w = init_bias
        initial_bias = 0
        G_s_loss = torch.tensor(0)
        #G_c_loss = torch.tensor(0)
        G_g_loss = torch.tensor(0)


        dfs = torch.tensor([ 1.0, 1.0, 1.0], device=device, dtype=torch.float, requires_grad=True)

        for i in range(epochs):
   
            if True:
                noise = torch.randn(batch_size,self.source.org_source_length).to(device) 
                bias = torch.zeros_like(noise).to(device)
                #if i < epochs/4:
                #bias = torch.randn(batch_size,self.source.org_source_length).to(device) / 4 
                #else:
                
                #bias = torch.randn(batch_size,self.source.org_source_length).to(device) 
                sw, sw0 = self.generator(noise,bias)
                with torch.no_grad():                
                    fake_gmr_out, fake_sim_out, fake_len_out = self.__discrete_gradient(sw)

                #print(fake_len_out)

                sw1 = sw * fake_sim_out
                G_s_loss = -torch.mean(sw1) 
                sw2 = sw * fake_gmr_out
                G_g_loss = -torch.mean(sw2) 
                sw3 = sw * fake_len_out
                G_l_loss = -torch.mean(sw3) * 10

                dsc_loss = torch.stack([G_g_loss,G_s_loss,G_l_loss])

                G_loss = torch.dot(dfs,dsc_loss) + torch.std(dsc_loss)
                #G_loss = G_g_loss + G_s_loss

                self.generator.zero_grad()
                G_loss.backward()
                #print('backward:')
                G_opt.step()
                scheduler.step()
                '''
                learning_rate = 0.1
                with torch.no_grad():
                    dfs += learning_rate * dfs.grad
                    dfs.grad = None                    
                    dfs[dfs < 0] = 0.1                
                '''

            gen_gmr_loss_history.append(G_g_loss.cpu().detach().numpy())
            #gen_com_loss_history.append(G_c_loss.cpu().detach().numpy())
            gen_sim_loss_history.append(G_s_loss.cpu().detach().numpy())
            #dis_loss_history.append(D_loss.cpu().detach().numpy())
            gen_len_loss_history.append(G_l_loss.cpu().detach().numpy())

            #pb.printProgress(+1,f'{i+1}/{epochs} epochs, beta:{dfs} Generator / grammar loss:{G_g_loss}  similarity loss:{G_s_loss}') #,   Discriminator grammar_loss:{D_loss}        ')
            #pb.printProgress(+1,'{}/{} epochs, beta:{}, grammar loss:{:.4f}  similarity loss:{:.4f} length loss:{:.4f}'.format(i+1,epochs,dfs,G_g_loss,G_s_loss,G_l_loss)) #,   Discriminator grammar_loss:{D_loss}        ')
            pb.printProgress(+1,'{}/{} epochs,grammar loss:{:.4f}  similarity loss:{:.4f} length loss:{:.4f}'.format(i+1,epochs,G_g_loss,G_s_loss,G_l_loss)) #,   Discriminator grammar_loss:{D_loss}        ')
            
            total_loss_history.append(torch.sum(dsc_loss).item())
            losses_std_history.append(torch.std(dsc_loss).item())

            
        self.generator.eval()
        #self.g_discriminator.discriminator.eval()
        if display:
            plt.figure(figsize=(12, 6))
            xs = np.arange(self.source.org_source_length)
            plt.bar(xs+0.0,sw0[0].cpu().detach().numpy(),label='before activation weights',width=0.2)
            plt.bar(xs+0.2,sw[0].cpu().detach().numpy(),label='after activation weights',width=0.2)
            plt.bar(xs+0.4,bias[0].cpu().detach().numpy(),label='bias weights',width=0.2)         
            plt.legend()        
            plt.show()

        return  {'gen_g_loss':gen_gmr_loss_history,'gen_s_loss':gen_sim_loss_history,'gen_l_loss':gen_len_loss_history,'total loss':total_loss_history,'losses std':losses_std_history} #,'dis_loss':dis_loss_history }

    def get_summary(self, count):
        #texts = []
        self.generator.cpu()
        self.generator.eval()
        #gen_length = len(self.source.story_peaks) + int(len(self.source.story_peaks)*self.frame_expansion_ratio)
        noise = torch.randn(count,self.source.org_source_length)
        bias = torch.zeros_like(noise)

        with torch.no_grad():
            sw,sw0 = self.generator(noise,bias)
            #sw,sw0 = self.generator(noise)

        max_score = 0
        max_sim = 0
        comp_rate = 0
        best_text = ""

        for p_txt in sw:
            gtext = self.__text_gen3(p_txt)
            text = ' '.join(gtext)
            #print('>>',text)
            sim_score = self.s_discriminator.similarity(text,self.source.org_text_emb)
            if sim_score > max_sim:
                best_text = text.strip()
                loss, out=self.g_discriminator.transfer_learning([text],train_for = False)
                max_score = out[0,1].item()
                comp_rate = 1 - len(best_text.split(' '))/self.source.org_source_length
                max_sim = sim_score
            #texts.append([text.strip(),out,sim_score])
        return best_text, max_score, max_sim, comp_rate

    def get_samples(self,count):
        self.generator.cpu()
        self.generator.eval()
        noise = torch.randn(count,self.source.org_source_length)
        bias = torch.zeros_like(noise)
        with torch.no_grad():
            sw,sw0 = self.generator(noise,bias)
        #samples = []
        best_text = ""
        max_score = 0
        for p_txt in sw:
            gtext = self.__text_gen3(p_txt)
            text = ' '.join(gtext).strip()
            #print(text)
            loss, out=self.g_discriminator.transfer_learning([text],train_for = False)
            sim_score = self.s_discriminator.similarity(text,self.source.org_text_emb)    
            comp_rate = 1 - len(text.split(' '))/self.source.org_source_length
            #samples.append((text,out[0,1].item(),sim_score,comp_rate))
            score = out[0,1].item() + sim_score + comp_rate
            if max_score < score:
                max_score = score
                best_text = text
        return [best_text for i in range(count)], max_score


#5. Experiment

## 비교 대상 요약 알고리즘 준비

In [51]:
def sam_wgan4(text, epochs=50, batch_size=32,display=False, retry = True, retry_count = 0):
    if retry_count > 7:
        raise Exception("Can't summarize the text")

    source = Source(text)
    source.set_key_rate(s_discriminator)
    summarizer = SAM_Summarizer(g_discriminator,s_discriminator)
    summarizer.ready(source)
    hist = summarizer.summarize(epochs,batch_size=2,learning_rate=5e-3,display=display)
    samples, max_score = summarizer.get_samples(batch_size)
    #print(samples)
    if retry and max_score < 4.1:
        print('max score:',max_score)
        return sam_wgan4(text, epochs+10, batch_size,display=display,retry_count=retry_count+1)
    return samples, max_score

In [ ]:
txt = """
옛날 어느 집에 귀여운 여자 아기가 태어났어요.
아기는 무럭무럭 자라서, 예쁘고 마음씨 고운 소녀가 되었어요.
"""
sam_wgan4(txt,epochs=100,display= True,retry = True)

In [ ]:
txt = """
서산시의회(의장 임재관) 가충순·이수의 의원이 (사)한국지역신문협회에서 수여하는 우수의정대상을 받았다. 가충순 의원과 이수의 의원은 16일 팔봉면 폰타나 리조트에서 열린 한국지역신문협회 하계 워크샵에서 지역사회 발전을 위해 활발한 의정활동을 펼친 공로를 인정받아 우수의정대상을 수상했다.
"""
sam_wgan4(txt,epochs=100,display= True,retry = False)

In [ ]:
full_text = """
옛날 어느 집에 귀여운 여자 아기가 태어났어요.
아기는 무럭무럭 자라서, 예쁘고 마음씨 고운 소녀가 되었어요.
그러던 어느날, 소녀의 어머니가 병이들어 그만 세상을 떠나고 말았어요.
소녀의 아버지는 홀로 남은 소녀가 걱정되었어요.
그래서 얼마 지나서 새어머니를 맞이했어요.
새어머니는 소녀보다 나이가 위인 두명의 딸을 데리고 왔어요.
그러나 새어머니와 언니들은 성질이 고약한 심술쟁이들이었어요.
새어머니는 소녀가 자기 딸들보다 예쁘고 착한게 못마땅했어요.
그런데 이번에는 아버지마저 돌아가셨어요.
소녀는 쓸고, 닦고, 하녀처럼 하루 종일 집안일을 도맡아 했어요.
집안일이 힘들어 지칠때면 난롯가에 앉아서 잠시 쉬곤 했어요.
그러던 어느날, 왕궁에서 무도회가 열렸어요.
신데렐라의 집에도 무도회 초대장이 왔어요.
새어머니는 언니들을 데리고 무도회장으로 떠났어요.
신데렐라도 무도회에 가고 싶었어요.
혼자 남은 신데렐라는 훌쩍훌쩍 울기 시작했어요.
그때 어디선가 마법사 할머니가 나타났어요.
신데렐라가 고개를 들어보니, 마법사 할머니가 빙그레 웃고 있었어요.
할머니는 소녀를 무도회에 보내줄테니 호박 한개와 생쥐 두마리, 도마뱀을 가지고 오라 했어요.
마법사 할머니가 이것들을 보면서 주문을 외웠어요.
그리고 지팡이로 호박을 건드리자, 호박이 화려한 황금마차로 변했어요.
이번에는 생쥐와 도마뱀을 건드렸어요.
그랬더니 생쥐는 흰말로, 도마뱀은 멋진 마부로 변했어요.
신데렐라의 낡은 옷은 구슬 장식이 반짝이는 예쁜 드레스로 바뀌었어요.
할머니는 신데렐라에게 반짝반짝 빛나는 유리구두를 신겨 주었어요.
그리고 밤 열두시가 되면 모든게 처음대로 돌아간다고 알려주었어요.
황금마차는 호박으로, 흰말은 생쥐로, 마부는 도마뱀으로 변하게 돼어요. 
그러니까 반드시 밤 열두시가 되기 전에 돌아와야 해요.
신데렐라는 황금마차를 타고 왕궁 무도회장으로 가서 멋진 왕자님을 만났어요.
왕자님도 아름다운 신데렐라에게 마음을 빼았겼어요.
왕자님은 무도회장에 모인 다른 아가씨들은 쳐다보지도 않고 신데렐라하고만 춤을 추었어요.
신데렐라는 왕자님과 춤을 추느라 시간 가는 줄도 몰랐어요.
어느덧 시간이 흘러 열두시가 되었어요. 
벽시계의 열두시를 알리는 종소리에 신데렐라는 화들짝 놀랐어요.
신데렐라가 허둥지둥 왕궁을 빠져나가는데, 유리구두 한짝이 벗겨졌어요.
하지만 구두를 주울 시간이 없었어요.
신데렐라를 뛰쫓아오던 왕자님은 층계에서 유리구두 한짝을 주웠어요.
왕자님은 유리구두를 가지고 임금님께 가서 말했어요.
이 유리구두의 주인과 결혼하겠어요.
그래서 신하들은 유리구두의 주인을 찾아 온 나라를 돌아다녔어요.
드디어 신데렐라의 집에까지 신하들이 도착했어요.
언니들은 발을 오므려도 보고, 구두를 늘려도 보았지만 한눈에 보기에도 유리구두는 너무 작았어요.
그때 신데렐라가 조용히 다가와 자기도 한번 신어보게 해달라고 부탁했어요.
신데렐라는 신하에게서 받은 유리구두를 신었어요.
유리구두는 신데렐라의 발에 꼭 맞았어요.
신하들은 신데렐라를 왕궁으로 데리고 갔어요.
그 뒤 신데렐라는 왕자님과 결혼하여 오래오래 행복하게 살았대요.
"""

In [ ]:
txt = """
옛날 어느 집에 귀여운 여자 아기가 태어났어요.
아기는 무럭무럭 자라서, 예쁘고 마음씨 고운 소녀가 되었어요.
"""
sam_wgan4(txt.strip(),display= True)


# EncoderDecoderModel

In [27]:
pre_trained_kobert_model_name='kykim/bert-kor-base'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [28]:
from transformers import EncoderDecoderModel, BertTokenizerFast
import torch

tokenizer = BertTokenizerFast.from_pretrained(pre_trained_kobert_model_name)

In [29]:
op = tokenizer('옛날 어느 집에 귀여운 여자 아기가 태어났어요.[SEP]아기는 무럭무럭 자라서 예쁘고 마음씨 고운 소녀가 되었어요.', return_tensors="pt",padding="max_length", truncation=True, max_length=64)
print(op)
print("Tokens (str)      : {}".format([tokenizer.convert_ids_to_tokens(s) for s in op['input_ids'].tolist()[0]]))
print("Tokens (int)      : {}".format(op['input_ids'].tolist()[0]))
print("Tokens (attn_mask): {}\n".format(op['attention_mask'].tolist()[0]))

{'input_ids': tensor([[    2, 17463, 14385, 14662, 15886, 14891, 17818, 33791, 13972,  2016,
             3, 35244,  4215,  8669,  8035,  8669, 19206,  8044, 17364, 14125,
          8472, 26268, 18857,  8048, 17292,  2016,     3,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

## dataset 만들기...???

In [ ]:

encoder_max_length = 64
decoder_max_length = 64
batch_size = 16 #4 # 64, 128
dataset_iterator = []
batch_counter = 0


org_sentences = np.array(nltk.sent_tokenize(full_text.strip()))
summary_text = []
for i in range(0,len(org_sentences),2):
    txt = org_sentences[i]
    txt2 = txt
    if i < len(org_sentences)-1:
        txt = txt +'[SEP]'+ org_sentences[i+1]
        txt2 += org_sentences[i+1]

    #text,g,s = sam_wgan3(txt2)
    #print(text,g,s)
    source = Source(txt2)
    source.set_key_rate(s_discriminator)

    batch = {}
    batch['input_ids'] = []
    batch['attention_mask'] = []
    batch['decoder_input_ids'] = []
    batch['decoder_attention_mask'] = []
    batch['labels'] = []
    batch['reward'] = []
    batch['combine_text'] = []

    percent = ("{0:.2f}").format(100 * ((i+2) / float(len(org_sentences))))
    print(f'{percent}% {i+2}/{str(len(org_sentences))}')

    for i in range(batch_size):
        text,h = source.get_random_text(0.5)
        loss, out= g_discriminator.transfer_learning([text],train_for = False)
        g = out[0,1].item()
        s = s_discriminator.similarity(text,source.org_text_emb)

        inputs = g_discriminator.tokenizer(txt, padding="max_length", truncation=True, max_length=encoder_max_length)
        outputs = g_discriminator.tokenizer(text, padding="max_length", truncation=True, max_length=decoder_max_length)

        rwd = (g + (s-0.5)*10 + 5)/10 * 5 - 3

        batch["input_ids"].append(inputs.input_ids)
        batch["attention_mask"].append(inputs.attention_mask)
        batch["decoder_input_ids"].append(outputs.input_ids)
        batch["decoder_attention_mask"].append(outputs.attention_mask)
        batch["labels"].append(outputs.input_ids.copy())
        batch["reward"].append(rwd)
        batch['combine_text'].append(text)

    dataset_iterator.append(batch)



# EncoderDecoder 학습

In [30]:
from transformers import EncoderDecoderModel, BertTokenizerFast

try:
    del model
    print('delete model')
except Exception as ex:
    pass

use_pretrained_model = False

if use_pretrained_model:
    model = EncoderDecoderModel.from_pretrained("/content/drive/MyDrive/GAN_ENDE/model")
    print('Load from pre-trained model...')
else:
    model = EncoderDecoderModel.from_encoder_decoder_pretrained(pre_trained_kobert_model_name, pre_trained_kobert_model_name) # initialize Bert2Bert from pre-trained checkpoints
    print('Initialize with ',pre_trained_kobert_model_name)

model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.eos_token_id = tokenizer.sep_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.encoder.vocab_size

model.config.max_length = 142
model.config.min_length = 56
model.config.no_repeat_ngram_size = 3
model.config.early_stopping = True
model.config.length_penalty = 2.0
model.config.num_beams = 4

N_EPOCHS = 100

Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertLMHeadModel: ['cls.seq_re

Initialize with  kykim/bert-kor-base


In [31]:
from transformers import BertTokenizer, AutoTokenizer, BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup

# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
        
criterion = nn.CrossEntropyLoss(ignore_index = tokenizer.pad_token_id)

In [32]:
def train2(model, iterator, optimizer, criterion, scheduler,clip):
    
    model.train()
    
    epoch_loss = 0
    optimizer.zero_grad()
    for i, batch in enumerate(iterator):

        input_ids = torch.tensor(batch["input_ids"]).to(device)
        attention_mask = torch.tensor(batch["attention_mask"]).to(device)
        decoder_input_ids= torch.tensor(batch["decoder_input_ids"]).to(device)
        decoder_attention_mask= torch.tensor(batch["decoder_attention_mask"]).to(device)
        labels= torch.tensor(batch["labels"]).to(device)

            
        outputs = model(input_ids=input_ids, attention_mask=attention_mask,
                        decoder_input_ids=decoder_input_ids, 
                        decoder_attention_mask=decoder_attention_mask,
                        labels=labels)

        loss = outputs.loss #* b_rewards
        loss.backward()
        ##print('loss',loss)
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        scheduler.step()
        
        epoch_loss += loss.item()
        ##print('epoch_loss',epoch_loss)
    return epoch_loss / len(iterator)

In [33]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [34]:
def generate_summary(text):
    # cut off at BERT max length 512
    sens = nltk.sent_tokenize(text)
    assert(len(sens) == 2)
    inputs = tokenizer(sens[0].strip()+'[SEP]'+sens[1].strip(), padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    model.to(device)
    model.eval()
    outputs = model.generate(input_ids, attention_mask=attention_mask).cpu().detach().numpy()[0]
    o=[]
    for token in outputs:
        if token == tokenizer.pad_token_id:
            break
        o.append(token)
    output_str = tokenizer.batch_decode([o], skip_special_tokens=True)[0]

    return output_str

In [45]:
documents = []
#documents.append(full_text)
for i in range(len(d2)):
    documents.append(' '.join(d2[i]['article_original']))

for i in range(len(d1)):
    documents.append(' '.join(d1[i]['article_original']))

In [47]:
documents[1]

'서산시의회(의장 임재관) 가충순·이수의 의원이 (사)한국지역신문협회에서 수여하는 우수의정대상을 받았다. 가충순 의원과 이수의 의원은 16일 팔봉면 폰타나 리조트에서 열린 한국지역신문협회 하계 워크샵에서 지역사회 발전을 위해 활발한 의정활동을 펼친 공로를 인정받아 우수의정대상을 수상했다. 지난해 6월 제7회 전국동시지방선거를 통해 등원한 두 의원은 산업건설위원회에서 열정적인 의정활동을 펼치고 있다. 가충순 의원은 5분발언, 행정사무감사, 시정질문을 통해 자동차 연비테스트 연구시설 유치, 천수만 염해피해 재발 방지, 서산시 대표 농산물 육성 등 지역의 크고 작은 문제를 개선하기 위해 노력하고 있다. 이수의 의원은 지난 행정사무감사에서 대산공단 기업 임원을 참고인으로 출석시켜 지역인재채용 및 관내업체·자재 활용을 확대할 것을 제안하며 기존 행정사무감사의 틀을 깨는 등 다양한 의정활동을 펼쳐나가고 있다. 가충순 의원은 "시의원이라면 마땅히 해야할 일을 한 것 뿐인데 상까지 주시니 몸 둘 바를 모르겠다"며 "항상 초심을 잊지 않고 지역 발전을 위해 최선을 다하겠다"고 소감을 밝혔다. 이수의 의원은 "믿고 뽑아주신 주민들을 위해 당연히 해야할 일을 했을 뿐인데 상까지 받게 돼 영광"이라며 "시민들이 자부심을 느낄 수 있는 지역사회를 만들어 나가기 위해 앞으로도 최선을 다 하겠다"고 말했다.'

In [36]:


import json

experiment_config = {}

if use_pretrained_model:
    # Opening JSON file
    f = open('/content/drive/MyDrive/GAN_ENDE/model/experiment_config.json','r')
    
    # returns JSON object as 
    # a dictionary
    experiment_config = json.load(f)
else:
    experiment_config['curren_doc_num'] = 0
    experiment_config['total_doc_num'] = len(documents)
    experiment_config['learing_time'] = 0

    with open('/content/drive/MyDrive/GAN_ENDE/model/experiment_config.json', 'w') as fp:
        json.dump(experiment_config, fp)

In [49]:
experiment_config['curren_doc_num'] = 2

In [50]:

encoder_max_length = 64
decoder_max_length = 64
batch_size = 32 #4 # 64, 128

CLIP = 1

best_valid_loss = float('inf')
model.to(device)

for cnt in range(experiment_config['curren_doc_num'],len(documents)):
    doc_start_time = time.time()
    doc = documents[cnt]
    dataset_iterator = []
    org_sentences = np.array(nltk.sent_tokenize(doc.strip()))
    first_summary = org_sentences[0] + ' ' + org_sentences[1]
    for i in range(0,len(org_sentences),2):
        txt = org_sentences[i]
        txt2 = txt
        if i < len(org_sentences)-1:
            txt +=  '[SEP]' + org_sentences[i+1]
            txt2 += org_sentences[i+1]

        batch = {}
        batch['input_ids'] = []
        batch['attention_mask'] = []
        batch['decoder_input_ids'] = []
        batch['decoder_attention_mask'] = []
        batch['labels'] = []
        try:
            samples, max_score = sam_wgan4(txt2,epochs=100,batch_size=batch_size)
            print('  max_score:',max_score)
            #for (text,g,s,l) in samples:
            for text in samples:
                inputs = g_discriminator.tokenizer(txt, padding="max_length", truncation=True, max_length=encoder_max_length)
                outputs = g_discriminator.tokenizer(text, padding="max_length", truncation=True, max_length=decoder_max_length)
                batch["input_ids"].append(inputs.input_ids)
                batch["attention_mask"].append(inputs.attention_mask)
                batch["decoder_input_ids"].append(outputs.input_ids)
                batch["decoder_attention_mask"].append(outputs.attention_mask)
                batch["labels"].append(outputs.input_ids.copy())
            dataset_iterator.append(batch)
        except Exception as ex:
            print(ex)

    # Total number of training steps is [number of batches] x [number of epochs]. 
    # (Note that this is not the same as the number of training samples).
    total_steps = len(dataset_iterator) * N_EPOCHS

    # Create the learning rate scheduler.
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0, # Default value in run_glue.py
                                                num_training_steps = total_steps)
    
    for epoch in range(N_EPOCHS):
        
        start_time = time.time()
        train_loss = train2(model, dataset_iterator, optimizer, criterion, scheduler,CLIP)
        end_time = time.time()
        
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
        #if valid_loss < best_valid_loss:
        #    best_valid_loss = valid_loss
        #    torch.save(model.state_dict(), 'tut6-model.pt')
        
        print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
        #print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
        print('\t---------------------------------------------')
        print('\tOrg text:',first_summary)
        sum_txt = generate_summary(first_summary)
        print('\tSummarized text:',sum_txt)
        print('\tCompression rate:{:4f}'.format(len(sum_txt)/len(first_summary)))
        print('\t---------------------------------------------')
        print('\tTarget text:',generate_summary('옛날 어느 집에 귀여운 여자 아기가 태어났어요.[SEP]아기는 무럭무럭 자라서 예쁘고 마음씨 고운 소녀가 되었어요.'))

    doc_end_time = time.time()
    model.save_pretrained("/content/drive/MyDrive/GAN_ENDE/model")
    experiment_config['curren_doc_num'] = cnt+1
    experiment_config['learing_time'] += doc_end_time - doc_start_time
    with open('/content/drive/MyDrive/GAN_ENDE/model/experiment_config.json', 'w') as fp:
        json.dump(experiment_config, fp)
    print(experiment_config)

--------------------------------------------------
지난 2004년 시작해 조선대 학생들의 대표적인 행사로 자리매김한 ‘조선대 국토대장정’이 지난 5일 해단식을 끝으로 일정을 마감했다. 조선대는 지난 5일 오후 서석홀 4층 대호전기홀에서 ‘2019학년도 조선대학교 제16기 국토대장정’ 해단식을 열었다.
--------------------------------------------------
Train... |||||||||||||||||||||| 100.0%   100/100 epochs,grammar loss:-0.2112  similarity loss:-0.1518 length loss:-0.2499
  max_score: 5.3661732853652575
--------------------------------------------------
국토대장정은 매년 여름방학에 조선대 학생들이 직접 우리 국토를 걸으면서 애국심을 느끼고 단체생활을 통해 협동심과 리더십 공동체 의식을 함양할 수 있도록 지원하는 행사다. 해마다 학생들에게 큰 호응을 받아 왔다.
--------------------------------------------------
Train... |||||||||||||||||||||| 100.0%   100/100 epochs,grammar loss:-0.0881  similarity loss:-0.0683 length loss:-0.1228
  max_score: 4.173073477716266
--------------------------------------------------
올해 국토대장정은 16번째 행사로 ‘조선대의 새로운 비상을 꿈꾸다’를 슬로건으로 진행됐다. 지난달 24일 출정식을 갖고 25일 새벽에 출발한 52명의 대원들은 총 10박 11일간 제주도 일대 총 321㎞를 행군했다.
--------------------------------------------------
Train..

KeyboardInterrupt: ignored

In [48]:
print(generate_summary('옛날 어느 집에 귀여운 여자 아기가 태어났어요.[SEP]아기는 무럭무럭 자라서 예쁘고 마음씨 고운 소녀가 되었어요.'))

서산시의회 ( 의장 임재관 ) 가충순 이수의 의원이 ( 사 ) 한국지역신문협회에서 수여하는 가충 의원은 16일 팔봉면 폰타나 한국지역기자협회 지역사회 위해 펼친 공로를 인정받아 수상했다.


In [ ]:
model.save_pretrained("/content/drive/MyDrive/GAN_ENDE/model")